In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# API 키 정보 로드
load_dotenv()

# 프로젝트 이름을 입력합니다.
logging.langsmith("chroma_VectorStores")

LangSmith 추적을 시작합니다.
[프로젝트명]
chroma_VectorStores


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import csv

file_path = "../data/csv_data/rental_data_with_null.csv"


def get_csv_headers(file_path):
    with open(file_path, mode="r", encoding="utf-8") as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # 첫 번째 줄(헤더) 가져오기
    return headers


headers = get_csv_headers(file_path)

# CSV 로더 생성
loader = CSVLoader(
    file_path=file_path,
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": headers,
    },
    # source_column="place",
    content_columns=headers,
    metadata_columns=[
        "place",
        "oneroom_half_year",
        "oneroom_year",
        "tworoom_half_year",
        "tworoom_year",
    ],
)
docs = loader.load()
print(docs[1].metadata)

{'source': '../data/csv_data/rental_data_with_null.csv', 'row': 1, 'place': '후문', 'oneroom_half_year': '', 'oneroom_year': '240', 'tworoom_half_year': '', 'tworoom_year': ''}


In [3]:
# metadata 값을 integer로 변환하는 함수
def convert_metadata_to_int(docs, columns_to_convert):
    """
    문서 리스트의 metadata 값을 integer로 변환하는 함수.
    
    Args:
        docs: CSVLoader에서 로드된 문서 리스트.
        columns_to_convert: integer로 변환할 열 이름 리스트.

    Returns:
        변환된 문서 리스트.
    """
    for doc in docs:
        for column in columns_to_convert:
            if column in doc.metadata:
                try:
                    # metadata 값을 integer로 변환
                    doc.metadata[column] = float(doc.metadata[column])
                except ValueError:
                    # 변환 실패 시 NaN 또는 다른 처리 (여기서는 0으로 설정)
                    doc.metadata[column] = float('nan')
    return docs

# 변환할 열 이름 리스트
columns_to_convert = [
    "oneroom_half_year",
    "oneroom_year",
    "tworoom_half_year",
    "tworoom_year",
]

# metadata 값을 integer로 변환
docs = convert_metadata_to_int(docs, columns_to_convert)

In [4]:
i = 1

for doc in docs[i:]:
    row = doc.page_content.split("\n")
    row_str = "<row>"
    for element in row:
        splitted_element = element.split(":")
        value = splitted_element[-1]
        col = ":".join(splitted_element[:-1])
        row_str += f"<{col}>{value.strip()}</{col}>"
    row_str += "</row>\n\n"

    docs[i].page_content = row_str
    i += 1
    # print(ret[i].page_content)
    # ret += row_str

ret = docs

In [5]:
print(ret[1])

page_content='<row><name>우리오피스텔</name><address>호서로125번길 25</address><contact>010-2335-6023</contact><price>년세 240</price><fee>X / 30만원</fee><options>풀옵션</options><gas_type>직접 문의</gas_type><comment>난방비, 수도세 개별 납부</comment><place>후문</place><oneroom_half_year></oneroom_half_year><oneroom_year>240</oneroom_year><tworoom_half_year></tworoom_half_year><tworoom_year></tworoom_year></row>

' metadata={'source': '../data/csv_data/rental_data_with_null.csv', 'row': 1, 'place': '후문', 'oneroom_half_year': nan, 'oneroom_year': 240.0, 'tworoom_half_year': nan, 'tworoom_year': nan}


In [6]:
# TextSplitter 테스트용 코드
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 각 문서에 파일 이름을 추가합니다.
for i in ret[1:]:
    i.metadata["filename"] = i.metadata["source"]

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=0) 

# splitted_docs = text_splitter.split_documents(ret[1:])
# print(len(splitted_docs))

In [7]:
# 문서를 예쁘게 출력하기 위한 도우미 함수
def pretty_print_docs(docs):
    for doc in docs:
        print("palce", doc.metadata["place"])
        print("oneroom_half_year", doc.metadata["oneroom_half_year"])
        print("oneroom_year", doc.metadata["oneroom_year"])
        print("tworoom_half_year", doc.metadata["tworoom_half_year"])
        print("tworoom_year", doc.metadata["tworoom_year"])
        print("\n=====================================\n")

In [8]:
print(ret[1:2])

[Document(metadata={'source': '../data/csv_data/rental_data_with_null.csv', 'row': 1, 'place': '후문', 'oneroom_half_year': nan, 'oneroom_year': 240.0, 'tworoom_half_year': nan, 'tworoom_year': nan, 'filename': '../data/csv_data/rental_data_with_null.csv'}, page_content='<row><name>우리오피스텔</name><address>호서로125번길 25</address><contact>010-2335-6023</contact><price>년세 240</price><fee>X / 30만원</fee><options>풀옵션</options><gas_type>직접 문의</gas_type><comment>난방비, 수도세 개별 납부</comment><place>후문</place><oneroom_half_year></oneroom_half_year><oneroom_year>240</oneroom_year><tworoom_half_year></tworoom_half_year><tworoom_year></tworoom_year></row>\n\n')]


In [9]:
from langchain_chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 저장할 경로 지정
DB_PATH = "./chroma_db"

# 문서를 디스크에 저장합니다. 저장시 persist_directory에 저장할 경로를 지정합니다.
persist_db = Chroma.from_documents(
    ret[1:],
    embeddings,
    persist_directory=DB_PATH,
    collection_name="rental_data_with_nan",
)

In [ ]:
from langchain_chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

# 저장할 경로 지정
DB_PATH = "./chroma_db"

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# # 디스크에서 문서를 로드합니다.
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="rental_data_with_null",
)

In [11]:
persist_db.get()

{'ids': ['0021e73e-0403-4364-a282-dbbbfb0ab02d',
  '04329f9e-cf92-4287-bdc4-855a0426b322',
  '053c8436-b790-4eaa-ba6c-7e8f3a6e7bb7',
  '086ec6f9-c820-4ea5-a206-bc186f4f7d40',
  '0eec6518-945d-4b18-b628-c0a1ef6952b5',
  '0fa2cfdb-abf3-45c7-9d24-2501fea24083',
  '1053b2e4-b0d2-4364-9671-68e04e47b3d0',
  '17ad227a-9471-43f2-9243-50b6b24d6e78',
  '19cf9144-49e9-4fec-8404-c2c36a0f2978',
  '2257961c-c6c8-4bf9-b3a8-caa996dfeeca',
  '238915fb-13e3-49bf-8648-52039c30a357',
  '258e5c66-3816-4b34-8905-700a56ebe86f',
  '266a3caa-7d17-4b57-b06a-c671e131aa6c',
  '273c5b3f-2081-4075-ac0a-d6d1a398eb9c',
  '2f3abf8a-9a71-44fa-a812-e7da1d1aa4de',
  '2f7ee792-d211-41f0-a27f-d9de6fb0a2f2',
  '374abb35-6205-45ea-aec5-b90f0b293a1a',
  '3f06e338-8477-404f-9e8a-1fc1ad5324f6',
  '3f3e6ebc-f1d2-4e73-9f1e-ad9a027837ae',
  '4052701b-03b2-4807-b6d1-1e7101058bdd',
  '40db536a-d359-4ede-a721-3a576bdf9346',
  '4275cc0e-9ed4-4e12-b87e-47988836f2e7',
  '497f7ff4-4211-4ca1-a22e-e21eb7de617a',
  '546a62b8-c9db-4367-9620-

In [ ]:
# db_retriever = persist_db.as_retriever(search_kwargs={"k": 5})
# a = db_retriever.invoke("300~400 중문 자취방 추천해줘")
# pretty_print_docs(a)

In [ ]:
# 기본 chroma_retriever를 생성합니다.
# chroma_retriever = persist_db.as_retriever(search_kwargs={"k": 3})

In [ ]:
ret[1].metadata

In [ ]:
# self-query retriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

# 메타데이터 필드 정보 생성
metadata_field_info = [
    AttributeInfo(
        name="place",
        description="The location information of the rental room. One of ['정문', '중문', '후문', '기숙사', '농가마트', '교육문화회관']",
        type="string",
    ),
    AttributeInfo(
        name="oneroom_half_year",
        description="The half-year rent price for a one-room unit.",
        type="integer",
    ),
    AttributeInfo(
        name="oneroom_year",
        description="The annual rent price for a one-room unit.",
        type="integer",
    ),
    AttributeInfo(
        name="tworoom_half_year",
        description="The half-year rent price for a two-room unit.",
        type="integer",
    ),
    AttributeInfo(
        name="tworoom_year",
        description="The annual rent price for a two-room unit.",
        type="integer",
    ),
]

In [ ]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o")

# 문서 내용 설명과 메타데이터 필드 정보를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.
prompt = get_query_constructor_prompt(
    "Brief summary of a rental room",  # 문서 내용 설명
    metadata_field_info,  # 메타데이터 필드 정보
)

# StructuredQueryOutputParser 를 생성
output_parser = StructuredQueryOutputParser.from_components()

# query_constructor chain 을 생성
query_constructor = prompt | llm | output_parser

In [ ]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor,  # 이전에 생성한 query_constructor chain 을 지정
    vectorstore=persist_db,  # 벡터 저장소를 지정
    structured_query_translator=ChromaTranslator(),  # 쿼리 변환기
    search_kwargs={"k": 5},  # 검색 옵션
)

In [10]:
result = retriever.invoke("년세 300~400 후문 자취방 추천해줘")
pretty_print_docs(result)

NameError: name 'retriever' is not defined

In [ ]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# retriever = SelfQueryRetriever.from_llm(
#     llm=llm,
#     vectorstore=persist_db,
#     document_contents="Brief summary of a rental room",
#     metadata_field_info=metadata_field_info,
#     enable_limit=True,  # 검색 결과 제한 기능을 활성화합니다.
#     search_kwargs={"k": 5},  # k 의 값을 2로 지정하여 검색 결과를 2개로 제한합니다.
# )

In [ ]:
# # # Self-query 검색
# result=retriever.invoke("농가마트 근처 자취방 추천해주세요")
# pretty_print_docs(result)